In [1]:
import os
import sys

sys.path.append("../")
print(os.getcwd())
print(sys.path)
from segmentation_scripts.filter_records import *

/Users/zleblanc/Informing-The-Third-World/periodical_augmentation/periodical-issue-segmentation/notebooks
['/Users/zleblanc/Informing-The-Third-World/periodical_augmentation/periodical-issue-segmentation/notebooks', '/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python39.zip', '/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9', '/opt/homebrew/Cellar/python@3.9/3.9.18_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/Users/zleblanc/.virtualenvs/htrc-periodicals-venv/lib/python3.9/site-packages', '../']


In [97]:
import pandas as pd
import os
df = pd.read_csv(os.path.join("..", "..", "..", "periodical-collection-curation", "HathiTrust-pcc-datasets", "datasets", "preidentified_periodicals_with_full_metadata.csv"))

In [98]:
df[0:1][['periodical_name', 'lowercase_periodical_name', 'publication_directory', 'volume_directory', 'record_url']]

,periodical_name,lowercase_periodical_name,publication_directory,volume_directory,record_url
0,La Documentation Arabe,la_documentation_arabe_,datasets/ht_ef_datasets/la_documentation_arabe...,inu_30000093395964,https://catalog.hathitrust.org/Record/007579658


In [99]:
subset_df = df[df.lowercase_periodical_name.str.contains('arab_observer', na=False)]

In [100]:
from datetime import datetime

def transform_annotated_dates(rows: pd.DataFrame) -> pd.DataFrame:
    """Transform metadata dates into mergeable dates."""
    date = rows.iloc[0].dates.replace('-', ' ').split(' ')
    day = date[1] if (len(date) == 3) and ('-' not in rows.iloc[0].dates) else '1'
    start_month = date[0]
    end_month = date[1] if (len(date) > 2) and ('-' in rows.iloc[0].dates) else start_month
    year = date[-1]

    start_date = f"{day} {start_month} {year}"
    end_date = f"{day} {end_month} {year}"
    rows['start_issue'] = datetime.strptime(start_date, '%d %B %Y')
    rows['end_issue'] = datetime.strptime(end_date, '%d %B %Y')
    return rows

def clean_annotated_df(annotated_df: pd.DataFrame) -> pd.DataFrame:
    """Clean and normalize dates in the annotated datasets that were created manually in Notion."""
    annotated_df.columns = [x.lower().replace(' ', '_') for x in annotated_df.columns]
    annotated_df['notes'] = annotated_df['notes'].fillna('')
    annotated_df = annotated_df.ffill()
    print(annotated_df.columns)
    annotated_df = annotated_df.groupby('dates', group_keys=False).apply(transform_annotated_dates, include_groups=True)
    return annotated_df

In [166]:
test_annotated_df = pd.read_csv("../datasets/annotated_datasets/arab_observer_annotated_dataset.csv")
cleaned_test_annotated_df = clean_annotated_df(test_annotated_df)

Index(['issue_number', 'dates', 'page_number', 'type_of_page',
       'original_volumes', 'notes'],
      dtype='object')


/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_87386/1098092194.py:23: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annotated_df = annotated_df.groupby('dates', group_keys=False).apply(transform_annotated_dates, include_groups=True)


In [167]:
subset_df = subset_df.rename(columns={'date': 'original_volumes'})

In [169]:
merged_annotated_df = pd.merge(cleaned_test_annotated_df,subset_df,  on='original_volumes', how='inner')

In [170]:
merged_annotated_df['original_volumes'].nunique()

23

In [171]:
merged_annotated_df.htid.unique()

array(['mdp.39015056038071', 'mdp.39015056038238', 'mdp.39015056038253',
       'mdp.39015056038246', 'mdp.39015056038063', 'mdp.39015056038410',
       'mdp.39015056038402', 'uc1.l0082986993', 'uc1.l0082987033',
       'uc1.l0063473003', 'uc1.l0063472831', 'mdp.39015056038089',
       'mdp.39015056038394', 'mdp.39015056038386', 'mdp.39015056038220',
       'mdp.39015056038378', 'mdp.39015056038212', 'inu.32000013025095',
       'inu.32000013025087', 'inu.32000013025079', 'inu.32000013025061',
       'inu.32000013025053', 'inu.32000013025046'], dtype=object)

In [69]:
test_path = os.path.join("..", "..", "..", "periodical-collection-curation", "HathiTrust-pcc-datasets", "datasets/ht_ef_datasets/afro_asian_and_world_affairs/volumes/mdp_39015035518128/mdp_39015035518128_individual_tokens.csv")
test_path

'../../../periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets/afro_asian_and_world_affairs/volumes/mdp_39015035518128/mdp_39015035518128_individual_tokens.csv'

In [172]:
ht_df = pd.read_csv("/Users/zleblanc/Informing-The-Third-World/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets/arab_observer_and_the_scribe/volumes/mdp_39015056038071/mdp_39015056038071_individual_tokens.csv")


In [173]:
ht_df = ht_df.rename(columns={'page': 'page_number', 'periodical_name': 'lowercase_periodical_name'})
ht_df[0:1][['lowercase_periodical_name', 'htid', 'record_url']]

,lowercase_periodical_name,htid,record_url
0,arab_observer_and_the_scribe,mdp.39015056038071,https://catalog.hathitrust.org/Record/000679918


In [177]:
subset_df.htid.nunique(), merged_annotated_df.htid.nunique()

(36, 23)

In [241]:
htid_value = ht_df.htid.unique()[0]
subset_merged_annotated_df = pd.merge(merged_annotated_df[merged_annotated_df.htid==htid_value], ht_df, on=['lowercase_periodical_name', 'htid', 'record_url', 'page_number'], how='outer')

In [244]:
merged_annotated_df[['publication_directory', 'volume_directory', 'original_volumes', 'start_issue', 'end_issue', 'htid', 'record_url']]

,publication_directory,volume_directory,original_volumes,start_issue,end_issue,htid,record_url
0,datasets/ht_ef_datasets/arab_observer_and_the_...,mdp_39015056038071,1962 Jan-Jun,1962-01-01,1962-01-01,mdp.39015056038071,https://catalog.hathitrust.org/Record/000679918
1,datasets/ht_ef_datasets/arab_observer_and_the_...,mdp_39015056038071,1962 Jan-Jun,1962-01-01,1962-01-01,mdp.39015056038071,https://catalog.hathitrust.org/Record/000679918
2,datasets/ht_ef_datasets/arab_observer_and_the_...,mdp_39015056038071,1962 Jan-Jun,1962-01-01,1962-01-01,mdp.39015056038071,https://catalog.hathitrust.org/Record/000679918
3,datasets/ht_ef_datasets/arab_observer_and_the_...,mdp_39015056038071,1962 Jan-Jun,1962-01-08,1962-01-08,mdp.39015056038071,https://catalog.hathitrust.org/Record/000679918
4,datasets/ht_ef_datasets/arab_observer_and_the_...,mdp_39015056038071,1962 Jan-Jun,1962-01-08,1962-01-08,mdp.39015056038071,https://catalog.hathitrust.org/Record/000679918
...,...,...,...,...,...,...,...
567,datasets/ht_ef_datasets/arab_observer_and_the_...,inu_32000013025046,no.327-331 1966,1966-10-17,1966-10-17,inu.32000013025046,https://catalog.hathitrust.org/Record/006064523
568,datasets/ht_ef_datasets/arab_observer_and_the_...,inu_32000013025046,no.327-331 1966,1966-10-24,1966-10-24,inu.32000013025046,https://catalog.hathitrust.org/Record/006064523
569,datasets/ht_ef_datasets/arab_observer_and_the_...,inu_32000013025046,no.327-331 1966,1966-10-24,1966-10-24,inu.32000013025046,https://catalog.hathitrust.org/Record/006064523
570,datasets/ht_ef_datasets/arab_observer_and_the_...,inu_32000013025046,no.327-331 1966,1966-10-31,1966-10-31,inu.32000013025046,https://catalog.hathitrust.org/Record/006064523


In [242]:
# Sort the DataFrame
# subset_merged_annotated_df = subset_merged_annotated_df.sort_values(by=['original_volumes', 'page_number'])

# Fill forward to propagate the last non-null date through NaNs temporarily
subset_merged_annotated_df['temp_date'] = subset_merged_annotated_df['dates'].ffill()

# Create a unique group ID based on when a new date occurs
subset_merged_annotated_df['issue_id'] = (subset_merged_annotated_df['temp_date'] != subset_merged_annotated_df['temp_date'].shift()).cumsum()
# Cast specific columns to desired types before filling
subset_merged_annotated_df['token'] = subset_merged_annotated_df['token'].astype(str)
subset_merged_annotated_df['notes'] = subset_merged_annotated_df['notes'].astype(str)
subset_merged_annotated_df['section'] = subset_merged_annotated_df['section'].astype(str)
subset_merged_annotated_df['pos'] = subset_merged_annotated_df['pos'].astype(str)
subset_merged_annotated_df['count'] = subset_merged_annotated_df['count'].astype(float)  # Or 
with pd.option_context('future.no_silent_downcasting', True):
	# Function to fill within each issue block
	def fill_issue_block(group):
		# Fill specific columns within the group
		group[['token', 'notes', 'section', 'pos']] = group[['token', 'notes', 'section', 'pos']].fillna('')
		group['count'] = group['count'].fillna(0)
		group['type_of_page'] = group['type_of_page'].fillna("content")
		
		# Forward-fill and backward-fill within the group
		group = group.ffill().bfill()
		
		return group

	# Apply the filling function to each group
	subset_merged_annotated_df = subset_merged_annotated_df.groupby('issue_id', group_keys=False).apply(fill_issue_block)

# Drop the temporary columns
subset_merged_annotated_df = subset_merged_annotated_df.drop(columns=['temp_date', 'issue_id'])

# Infer data types after filling
subset_merged_annotated_df = subset_merged_annotated_df.infer_objects()

/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_87386/4046917960.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  subset_merged_annotated_df = subset_merged_annotated_df.groupby('issue_id', group_keys=False).apply(fill_issue_block)


In [230]:
subset_merged_annotated_df.dates.value_counts()

dates
June 25 1962        12986
March 19 1962       12918
March 12 1962       12453
January 29 1962     12322
January 22 1962     12039
March 26 1962       11992
January 1 1962      11929
May 14 1962         11891
April 23 1962       11833
June 18 1962        11832
February 19 1962    11766
April 30 1962       11690
March 5 1962        11665
February 26 1962    11623
April 16 1962       11496
June 11 1962        11488
January 15 1962     11354
February 12 1962    11346
February 5 1962     11299
April 2 1962        11257
May 28 1962         10852
May 21 1962         10840
June 4 1962         10721
May 7 1962          10708
April 9 1962        10561
January 8 1962      10034
Name: count, dtype: int64

In [231]:
# # Sort the DataFrame
# subset_merged_annotated_df = subset_merged_annotated_df.sort_values(by=['original_volumes', 'page_number'])

# # Ensure you are working with a copy
# subset_merged_annotated_df = subset_merged_annotated_df.copy()

# # Replace nulls in 'type_of_page' with 'content'
# subset_merged_annotated_df.loc[subset_merged_annotated_df['type_of_page'].isnull(), 'type_of_page'] = 'content'

# # Fill NaNs in specific columns with appropriate values
# subset_merged_annotated_df[['token', 'notes', 'section', 'pos']] = subset_merged_annotated_df[['token', 'notes', 'section', 'pos']].fillna('')

# # Fill NaNs in 'count' column with 0
# subset_merged_annotated_df['count'] = subset_merged_annotated_df['count'].fillna(0)

# # Forward-fill and backward-fill without downcasting
# subset_merged_annotated_df = subset_merged_annotated_df.ffill().bfill()

# # Infer data types after filling
# subset_merged_annotated_df = subset_merged_annotated_df.infer_objects()

In [232]:
def cut_vols(rows):
    # Make a copy to ensure original data is preserved
    final_rows = rows.copy()

    # Filter rows up to the last `end_of_issue` page_number, if present
    if 'end_of_issue' in final_rows['type_of_page'].values:
        end_page_number = final_rows[final_rows['type_of_page'] == 'end_of_issue']['page_number'].tolist()[-1]
        final_rows = final_rows[final_rows['page_number'] <= end_page_number]

    # Determine the first page_number to include based on `cover_page` or fallback to `toc`
    if 'cover_page' in final_rows['type_of_page'].values:
        first_page_number = final_rows[final_rows['type_of_page'] == 'cover_page']['page_number'].values[0]
    elif 'toc' in final_rows['type_of_page'].values:
        first_page_number = final_rows[final_rows['type_of_page'] == 'toc']['page_number'].values[0]
    else:
        first_page_number = 0  # Default to the beginning if neither `cover_page` nor `toc` is present

    # Exclude rows before the determined first page_number
    final_rows = final_rows[final_rows['page_number'] > first_page_number - 1]

    # Remove page_numbers within duplicate ranges, if `duplicates` is present
    if 'duplicates' in final_rows['type_of_page'].values:
        duplicates_range = final_rows[final_rows['type_of_page'] == 'duplicates']['notes'].values[0]
        start, end = map(int, duplicates_range.split('-'))
        final_rows = final_rows[~final_rows['page_number'].between(start, end)]

    return final_rows

In [233]:
test = subset_merged_annotated_df.copy()
test = test.sort_values(by=['original_volumes', 'page_number'])

In [234]:
test2 = test.groupby('dates', group_keys=False).apply(cut_vols, include_groups=True)

/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_87386/2002313983.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test2 = test.groupby('dates', group_keys=False).apply(cut_vols, include_groups=True)


In [235]:
len(test), len(test2), len(subset_merged_annotated_df)

(300895, 300791, 300895)

In [236]:
test2.type_of_page.value_counts()

type_of_page
content         293243
toc               6502
cover_page        1045
end_of_issue         1
Name: count, dtype: int64

In [237]:
subset_merged_annotated_df.type_of_page.value_counts()

type_of_page
content         293332
toc               6502
cover_page        1045
duplicates          14
scanner_page         1
end_of_issue         1
Name: count, dtype: int64

In [162]:
test2.dates.value_counts()

dates
June 25 1962        12986
April 23 1962         611
May 14 1962           432
May 7 1962            355
April 30 1962         351
June 11 1962          351
June 18 1962          348
January 1 1962        319
January 22 1962       300
January 15 1962       285
March 12 1962         281
February 12 1962      280
March 26 1962         277
February 19 1962      276
March 19 1962         276
February 26 1962      265
April 2 1962          256
May 21 1962           245
January 8 1962        239
May 28 1962           237
February 5 1962       232
April 9 1962          232
March 5 1962          225
April 16 1962         213
January 29 1962       205
June 4 1962           122
Name: count, dtype: int64

In [9]:
from thefuzz import fuzz, process
from rich.console import Console
console = Console()

annotated_datasets_output_path = os.path.join('..', 'datasets', 'annotated_datasets')
for _, _, files in tqdm(os.walk(annotated_datasets_output_path)):
	for f in files:
		if (f.endswith('.csv')):
			console.print(f'Processing {f}...', style="magenta")
			if 'freedomways' in f:
				continue
			# Create file name and join directories for files
			file_name = f.rsplit('.', 1)[0]
			filenames = '_'.join([fi for fi in file_name.split('_') if not fi.isdigit()])
			filenames = filenames.replace('_annotated_dataset', '')
			appended_data = False
			final_dir = ''
			print(filenames)
			choices = df.lowercase_periodical_name.unique().tolist()
			fuzzy_matches = process.extract(filenames, choices, scorer=fuzz.token_sort_ratio, limit=5)
			top_match = fuzzy_matches[0]
			if top_match[1] == 100:
				print(f'Fuzzy match found: {top_match[0]}')
				# print(df[df.lowercase_periodical_name == top_match[0]])
				final_dir = df[df.lowercase_periodical_name == top_match[0]].iloc[0]['publication_directory']
				appended_data = True
				annotated_df = pd.read_csv(os.path.join(annotated_datasets_output_path, f))
				annotated_df.Dates = annotated_df.Dates.str.replace('Decmeber', 'December')
				annotated_df.Dates = annotated_df.Dates.str.replace('Summer', 'July')
				cleaned_annotated_df = clean_annotated_df(annotated_df)
				print(cleaned_annotated_df.columns)
			else:
				print(f'No fuzzy match found for {filenames}')

0it [00:00, ?it/s]

Processing freedomways_annotated_dataset.csv...

Processing solidarity_annotated_dataset.csv...

solidarity
No fuzzy match found for solidarity


Processing afro_asian_peoples_annotated_dataset.csv...

...

/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_87386/2969925928.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  annotated_df = annotated_df.fillna(method='ffill')
/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_87386/2969925928.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annotated_df = annotated_df.groupby('dates').apply(transform_annotated_dates)


Processing tricontinental_annotated_dataset.csv...

tricontinental
Fuzzy match found: tricontinental
Index(['issue_number', 'dates', 'page_number', 'type_of_page',
       'original_volumes', 'notes'],
      dtype='object')


/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_87386/2969925928.py:20: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  annotated_df = annotated_df.fillna(method='ffill')
/var/folders/fc/c_5vvt992871rr8qbqy645340000gq/T/ipykernel_87386/2969925928.py:22: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  annotated_df = annotated_df.groupby('dates').apply(transform_annotated_dates)
1it [00:00,  2.34it/s]

Index(['issue_number', 'dates', 'page_number', 'type_of_page',
       'original_volumes', 'notes', 'start_issue', 'end_issue'],
      dtype='object')


In [7]:
		dir_list = [subdir.replace(extracted_features_output_dir, '') for subdir, _, _ in os.walk(extracted_features_output_dir)]
		subset_dir_list = [d for d in dir_list if d.count(os.sep) == 1]
		# Read the annotated mapping file
		annotated_mapping_output_path = os.path.join('..', 'datasets', 'mapping_files', 'annotation_metadata_mapping.csv')
		annotated_mapping_df = pd.read_csv(annotated_mapping_output_path)
		dfs = []

		# Iterate over all metadata files
		metadata_output_path = os.path.join('..', 'datasets', 'metadatas')
		for _, _, files in tqdm(os.walk(metadata_output_path)):
			for f in files:
				if f.endswith('.csv'):
					console.print(f'Processing {f}...', style="magenta")
					# Create file name and join directories for files
					file_name = f.rsplit('.', 1)[0]
					filenames = '_'.join([fi for fi in file_name.split('_') if not fi.isdigit()])
					appended_data = False
					final_dir = ''
					for dir_name in subset_dir_list:
						fuzziness = fuzz.ratio(filenames.replace('/', '').replace('_', ' '), dir_name.replace('/', '').replace('_', ' '))
						if fuzziness > 80:
							console.print(f"Found a match for {file_name} with {filenames.replace('/', '').replace('_', ' ')} and {dir_name.replace('/', '').replace('_', ' ')} of {fuzziness} fuzzy ratio.", style="bold green")
							appended_data = True
							final_dir = dir_name
							df = pd.DataFrame([{
								'local_dir': dir_name.split('/')[-1],
								'file_name': filenames,
								'fuzzy_ratio': fuzziness,
								'metadata_file': f,
								'final_dir': final_dir
							}])
							dfs.append(df)
							break
					if appended_data == False:
						# turn subdir list into enumerated dictionary
						processed_subset_dir_list = {i: d for i, d in enumerate(subset_dir_list)}
						console.print("No match found for", filenames.replace('/', '').replace('_', ' '), "in the subset directories. Please select the correct directory from the list below:", style="bold red")
						console.print(processed_subset_dir_list, style="bold blue")
						selected_dir = int(input("Please select the correct directory: "))
						final_dir = processed_subset_dir_list[selected_dir]
						df = pd.DataFrame([{
							'local_dir': final_dir.split('/')[-1],
							'file_name': filenames,
							'fuzzy_ratio': None,
							'metadata_file': f,
							'final_dir': final_dir
						}])
						dfs.append(df)

,link,htid,date,original_source,record_url,periodical_name,publication_type,id,metadata_schema_version,enumeration_chronology,...,is_part_of,last_rights_update_date,pub_place_ld,pub_place,main_entity_of_page,publisher_ld,publisher,lowercase_periodical_name,publication_directory,volume_directory
0,https://hdl.handle.net/2027/inu.30000093395964,inu.30000093395964,NaN,\n Indiana University\n ...,https://catalog.hathitrust.org/Record/007579658,La Documentation Arabe,Information Bulletin,inu.30000093395964,https://schemas.hathitrust.org/EF_Schema_Metad...,NaN,...,"{'id': 'http://www.worldcat.org/oclc/6101458',...",20180507.0,{'id': 'http://id.loc.gov/vocabulary/countries...,Egypt,['https://catalog.hathitrust.org/Record/007579...,"{'id': ""http://catalogdata.library.illinois.ed...",Agence d'information du Moyen-Orient.,la_documentation_arabe_,../datasets/ht_ef_datasets/la_documentation_ar...,inu_30000093395964
1,https://hdl.handle.net/2027/inu.30000093395972,inu.30000093395972,"v.2,no.1-23 1963",\n Indiana University\n ...,https://catalog.hathitrust.org/Record/007579658,La Documentation Arabe,Information Bulletin,inu.30000093395972,https://schemas.hathitrust.org/EF_Schema_Metad...,"v.2,no.1-23 1963",...,"{'id': 'http://www.worldcat.org/oclc/6101458',...",20180507.0,{'id': 'http://id.loc.gov/vocabulary/countries...,Egypt,['https://catalog.hathitrust.org/Record/007579...,"{'id': ""http://catalogdata.library.illinois.ed...",Agence d'information du Moyen-Orient.,la_documentation_arabe_,../datasets/ht_ef_datasets/la_documentation_ar...,inu_30000093395972
2,https://hdl.handle.net/2027/inu.30000117844955,inu.30000117844955,no.75-78 1964-1965,\n Indiana University\n ...,https://catalog.hathitrust.org/Record/008569258,Afrique Documents,Information Bulletin,inu.30000117844955,https://schemas.hathitrust.org/EF_Schema_Metad...,no.75-78 1964-1965,...,"{'id': 'http://www.worldcat.org/oclc/5969711',...",20180419.0,{'id': 'http://id.loc.gov/vocabulary/countries...,Senegal,['https://catalog.hathitrust.org/Record/008569...,{'id': 'http://catalogdata.library.illinois.ed...,Afrique Documents,afrique_documents,../datasets/ht_ef_datasets/afrique_documents/v...,inu_30000117844955
3,https://hdl.handle.net/2027/inu.30000117844948,inu.30000117844948,no.81-83 1965,\n Indiana University\n ...,https://catalog.hathitrust.org/Record/008569258,Afrique Documents,Information Bulletin,inu.30000117844948,https://schemas.hathitrust.org/EF_Schema_Metad...,no.81-83 1965,...,"{'id': 'http://www.worldcat.org/oclc/5969711',...",20180419.0,{'id': 'http://id.loc.gov/vocabulary/countries...,Senegal,['https://catalog.hathitrust.org/Record/008569...,{'id': 'http://catalogdata.library.illinois.ed...,Afrique Documents,afrique_documents,../datasets/ht_ef_datasets/afrique_documents/v...,inu_30000117844948
4,https://hdl.handle.net/2027/inu.30000117844963,inu.30000117844963,no.84-89 1966,\n Indiana University\n ...,https://catalog.hathitrust.org/Record/008569258,Afrique Documents,Information Bulletin,inu.30000117844963,https://schemas.hathitrust.org/EF_Schema_Metad...,no.84-89 1966,...,"{'id': 'http://www.worldcat.org/oclc/5969711',...",20180419.0,{'id': 'http://id.loc.gov/vocabulary/countries...,Senegal,['https://catalog.hathitrust.org/Record/008569...,{'id': 'http://catalogdata.library.illinois.ed...,Afrique Documents,afrique_documents,../datasets/ht_ef_datasets/afrique_documents/v...,inu_30000117844963
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
534,https://babel.hathitrust.org/cgi/pt?id=inu.300...,inu.30000125592216,v.6 1963,Indiana University,https://catalog.hathitrust.org/Record/000507426,Scribe,Information Bulletin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,https://babel.hathitrust.org/cgi/pt?id=mdp.390...,mdp.39015070444719,"v.7-8 no.1-5, 1-3 1963-1964",University of Michigan,https://catalog.hathitrust.org/Record/000507426,Scribe,Information Bulletin,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
536,https:/

In [2]:
os.path.exists("/Users/zleblanc/Informing-The-Third-World/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets")

True

In [6]:
import os

# Define the relative path from the current working directory
relative_path = os.path.join("..", "periodical-collection-curation", "HathiTrust-pcc-datasets", "datasets", "ht_ef_datasets")

# Compute the absolute path
absolute_path = os.path.abspath(relative_path)

# Check if the path exists
path_exists = os.path.exists(absolute_path)

print("Relative path:", relative_path)
print("Absolute path:", absolute_path)
print("Path exists:", path_exists)

Relative path: ../periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets
Absolute path: /Users/zleblanc/Informing-The-Third-World/periodical_augmentation/periodical-issue-segmentation/periodical-collection-curation/HathiTrust-pcc-datasets/datasets/ht_ef_datasets
Path exists: False


In [7]:
os.path.exists("../datasets")

True

In [16]:
relative_extracted_features_path = os.path.join("..", "..", "..", "periodical-collection-curation", "HathiTrust-pcc-datasets", "datasets", "ht_ef_datasets")
print(os.path.exists(relative_extracted_features_path))

True


In [19]:
dir_list = [subdir.replace(relative_extracted_features_path, '') for subdir, _, _ in os.walk(relative_extracted_features_path)]

In [21]:
# subset to only paths with one level of depth
subset_dir_list = [d for d in dir_list if d.count(os.sep) == 1]

In [22]:
subset_dir_list

['/liberator',
 '/spearhead',
 '/presence_africaine',
 '/arab_observer_and_the_scribe',
 "/bulletin_d’information__bureau_d'information_du_gouvernement_révolutionnaire_provisoire_de_la_république_du_sud_viet-nam_à_paris",
 '/news_from_xinhua_news_agency,_chinadaily_bulletin',
 '/oau_bulletin',
 '/black_news',
 '/lotus',
 '/tricontinental',
 '/afro_asian_and_world_affairs',
 '/afro_asian_peoples',
 '/la_documentation_arabe_',
 '/nahdat_ifriqiyah_african_renaissance',
 '/freedomways',
 '/direct_from_cuba',
 '/lsm_news__liberation_support_movement,_information_center',
 '/negro_digest',
 '/black_world',
 '/asia_and_africa_today',
 '/afro_asian_women',
 '/solidarity_swapo',
 '/afrique_documents',
 '/mozambican_revolution',
 '/al-nashrah_al-ifrīqīyah_al-āsyawīyah',
 '/solidarity_aapso',
 '/afro_asian_bulletin',
 '/im_information_bulletin__mozambique_information_agency',
 '/révolution_africaine']